# **MAAS User Interface**

Este notebook detalla la interfaz a utilizar para ejecutar MAAS. Funciona en Google Colab y en JupyterLab.

### JupyterLab:
- Se preguntará si gusta grabar desde el micrófono, de no ser así se solicitará el nombre de un archivo WAV a 44100 Hz, colocado en /samples/
- Se preguntará si gusta guardar el CSV generado con MAAS_filter() en /database/
- Funciona out of the box, siempre que tenga toda la paquetería requerida (numpy, IPython, soundfile, scipy, matplotlib, sounddevice, ipython-autotime)

### Colab:
- Puede utilizar el micrófono y grabar (si la sesión es nueva, puede que requiera de dos intentos por cuestiones de permisos)
- No se guardará ningún CSV
- Debe cargar la base de datos a la raíz de la sesión (\[/content\]/*.csv) para realizar la comparación
- Debe cargar el módulo MAAS_Filters.ipynb a la raíz de la sesión (\[/content\]/MAAS_Filters.ipynb), que se incluye en el GitHub

## Autores
- Diego Casta - 35705
- Mauricio Alcántar - 35860
- David Santana - 35967

## **Librerías**

In [ ]:
if IN_COLAB:
  !pip install ipython-autotime

In [ ]:
import numpy as np
from io import BytesIO
from base64 import b64decode
try:
    from google.colab import output
except ImportError:
    pass
from IPython.display import Javascript
import IPython.display as ipd
import io
import soundfile as sf
from scipy import signal as sig
from scipy.io import wavfile as wav
from os import listdir
import sounddevice as sd
import matplotlib.pyplot as plt
import math

In [ ]:
IN_COLAB = None
if 'google.colab' in str(get_ipython()):
  IN_COLAB = True
  print('Ejecutándose en Colab')
else:
  IN_COLAB = False
  print('Ejecutándose localmente')

In [ ]:
%load_ext autotime

## **Lectura de archivo**

In [ ]:
RECORD = None if not IN_COLAB else """
const sleep  = time => new Promise(resolve => setTimeout(resolve, time));

const b2text = blob => new Promise(resolve => {
  const reader = new FileReader();
  reader.onloadend = e => resolve(e.srcElement.result);
  reader.readAsDataURL(blob);
});

var record = time => new Promise(async resolve => {
  const stream = await navigator.mediaDevices.getUserMedia({ audio: true });
  const audioContext = new AudioContext({ sampleRate: 44100 });
  const source = audioContext.createMediaStreamSource(stream);
  const recorder = new MediaRecorder(stream);
  let chunks = [];

  recorder.ondataavailable = e => chunks.push(e.data);
  alert("Presiona 'aceptar' para empezar la grabación.");
  recorder.start();
  await sleep(time);

  recorder.onstop = async () => {
    const blob = new Blob(chunks, { type: 'audio/ogg; codecs=opus' });
    const arrayBuffer = await blob.arrayBuffer();
    const audioBuffer = await audioContext.decodeAudioData(arrayBuffer);

    // Convert to WAV format
    const wavBlob = encodeWAV(audioBuffer, 44100, 16);
    const wavText = await b2text(wavBlob);
    resolve(wavText);

    // Clean up
    stream.getTracks().forEach(track => track.stop());
    audioContext.close();
  };

  recorder.stop();
});

// WAV encoding function
function encodeWAV(audioBuffer, sampleRate, bitDepth) {
  const numChannels = audioBuffer.numberOfChannels;
  const length = audioBuffer.length * numChannels * (bitDepth / 8);
  const buffer = new ArrayBuffer(44 + length);
  const view = new DataView(buffer);

  // WAV header
  writeString(view, 0, 'RIFF');
  view.setUint32(4, 36 + length, true);
  writeString(view, 8, 'WAVE');
  writeString(view, 12, 'fmt ');
  view.setUint32(16, 16, true);  // Format chunk length
  view.setUint16(20, 1, true);   // PCM format
  view.setUint16(22, numChannels, true);
  view.setUint32(24, sampleRate, true);
  view.setUint32(28, sampleRate * numChannels * (bitDepth / 8), true); // Byte rate
  view.setUint16(32, numChannels * (bitDepth / 8), true); // Block align
  view.setUint16(34, bitDepth, true); // Bits per sample
  writeString(view, 36, 'data');
  view.setUint32(40, length, true);

  // Write PCM samples
  let offset = 44;
  for (let channel = 0; channel < numChannels; channel++) {
    const data = audioBuffer.getChannelData(channel);
    for (let i = 0; i < data.length; i++, offset += 2) {
      const sample = Math.max(-1, Math.min(1, data[i]));
      view.setInt16(offset, sample < 0 ? sample * 0x8000 : sample * 0x7FFF, true);
    }
  }

  return new Blob([view], { type: 'audio/wav' });
}

function writeString(view, offset, string) {
  for (let i = 0; i < string.length; i++) {
    view.setUint8(offset + i, string.charCodeAt(i));
  }
}
"""

def record_lab(duration=5, sample_rate=44100, channels=1):
    print("Grabando...")
    recording = sd.rec(int(duration * sample_rate), samplerate=sample_rate, channels=channels, dtype='int16')
    sd.wait()
    print("Grabación terminada.")
    return recording, sample_rate

def record(sec=10):
  if (IN_COLAB):
    output.eval_js(RECORD)
    sec += 1
    s = output.eval_js('record(%d)' % (sec*1000))
    print("Grabación terminada")
    return io.BytesIO(b64decode(s.split(',')[1]))
  else:
    recording, sample_rate = record_lab(duration=sec)
    byte_io = io.BytesIO()
    wav.write(byte_io, sample_rate, recording)
    byte_io.seek(0)
    return byte_io

In [ ]:
FILENAME = '' if IN_COLAB else input("Presiona ENTER para grabar o introduce el nombre de un archivo WAV colocado en /samples/")

In [ ]:
audio = record(10) if FILENAME == '' else open('/samples/' + FILENAME, 'rb')

In [ ]:
ipd.display(ipd.Audio(audio.read()))

In [ ]:
audio.seek(0)
data, sr = sf.read(audio)
if (data.ndim == 2): data = data.mean(axis=1).astype(data.dtype)
print("Sample rate is {0}Hz".format(sr))

In [ ]:
if (IN_COLAB):
  %run /content/MAAS_Filters.ipynb
else:
  %run ./MAAS_Filters.ipynb

In [ ]:
SHOULD_SAVE = False if IN_COLAB else input('¿Guardar CSV? [y/N]').lower() == 'y'

In [ ]:
filtered = MAAS_filter(data, sr, None if (not SHOULD_SAVE) else './database/' + audio + '.csv')

In [ ]:
filtered

In [ ]:
def LinearRegressionMAAS(timepairsRec,timepairsTest,details):
  timepairsRec = np.array(timepairsRec)
  timepairsTest = np.array(timepairsTest)
  n = len(timepairsRec)
  xi = timepairsRec
  yi = timepairsTest
  xi2=xi**2
  yi2=yi**2
  xiyi=xi*yi
  table=np.transpose([xi,yi,xi2,yi2,xiyi])
  xiS=np.sum(xi)
  yiS=np.sum(yi)
  xi2S=np.sum(xi2)
  yi2S=np.sum(yi2)
  xiyiS=np.sum(xiyi)
  sumApp=([xiS,yiS,xi2S,np.around(yi2S,decimals=2),xiyiS])  # Creacion de la tabla de sumatorias

  # Paso 1. Calculo de a1
  a1=(n*(xiyiS)-(xiS*yiS))/(n*xi2S-(xiS**2))
  xProm=xiS/n
  yProm=yiS/n

  # Paso 2. Calculo de a0
  a0=yProm-(a1*xProm)

  # Coeficiente de correlacion
  r=(n*xiyiS - (xiS*yiS))/((math.sqrt(n*(xi2S)-(xiS**2)))*(math.sqrt(n*(yi2S)-(yiS**2))))

  # Desviacion estandar
  sig=math.sqrt(((n*(yi2S))-(yiS**2))/(n*(n-1)))

  # Graficacion
  yG= a1*xi + a0
  plt.plot(xi,yG,'bo-')
  plt.plot(xi,yi,'ko')
  plt.grid()
  plt.title("Window")
  plt.xticks(xi)
  plt.show()
  print("r =",np.around(r,decimals=6),"\nr^2 =",np.around(r**2,decimals=6))
  if(details):
    print("a1 =",np.around(a1,decimals=6))
    print("a0 =",np.around(a0,decimals=6))
    print("y =",np.around(a1,decimals=4),"x +",np.around(a0,decimals=4))
    print("Desviación estándar:",np.around(sig,decimals=4))
    print()
  return (r)


In [ ]:
def SlidingWindowRegression(recording_song,test_song,step_size, silent = True):

  n = len(recording_song)
  m = test_song.shape[1]
  r_arr = []
  count = 0
  for i in range(0, m-n+1, step_size):
   current = test_song[:,i:i+n]
   timepairsRec = []
   timepairsTest = []
   for j in range(current.shape[1]):

      if(abs(current[1,j] - recording_song[1][j]) < 5):

        timepairsRec.append(recording_song[0][j])
        timepairsTest.append(current[0,j])

   count += 1

   if len(timepairsRec) > 1 and len(timepairsTest) > 1:
     r = (LinearRegressionMAAS(timepairsRec,timepairsTest,False))**2
     r_arr = np.append(r_arr,r)
     if (not silent): print(f'Window {count} done successfully.')

   else:
     if (not silent): print(f'Window {count} done, no pairs found.')

  if (np.any(r_arr)):
    return(np.max(r_arr))

  else:
    if (not silent): print("Zero matches.\n")

    return(0)

In [ ]:
def get_database():
    filenames = listdir("/content/" if IN_COLAB else "./database/")
    return [ filename for filename in filenames if filename.endswith( ".csv" ) ]

In [ ]:
def compare(filtered_song, db_obj, silent = True):
    test_song = np.transpose(np.genfromtxt(("/content/" if IN_COLAB else "./database/") + db_obj, delimiter=",", skip_header=1))
    
    #test_song is an np array
    #filtered is a python list
    
    n = len(filtered_song[1])
    
    # SlidingWindowRegression already squares 'r'
    r2 = SlidingWindowRegression(filtered_song,test_song,75,silent)
    
    if (r2):
      if (not silent): print("Best R^2:", r2)
    else:
      if (not silent): print(f"Zero correlation found for {db_obj}.")
    return r2

In [ ]:
database = get_database()
results = {}
for SONG in database:
    results[SONG] = compare(filtered, SONG)

print(f"La canción identificada fue {max(results, key=results.get)}")